In [ ]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [ ]:
!pip install torch==1.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x254c000 @  0x7fa2c983b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0+cu113 requires torch==1.12.0, but you have torch 1.10.0 which is incomp

In [ ]:
%cd /content/
!git clone https://github.com/kikirizki/PocketNet
!gdown 1sDnU-DuotD0nkJ_1AitlqZGu1Torwifu

/content
Cloning into 'PocketNet'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 622 (delta 32), reused 30 (delta 27), pack-reused 576
Receiving objects: 100% (622/622), 73.18 MiB | 36.55 MiB/s, done.
Resolving deltas: 100% (346/346), done.
Downloading...
From: https://drive.google.com/uc?id=1sDnU-DuotD0nkJ_1AitlqZGu1Torwifu
To: /content/295672backbone.pth
100% 4.24M/4.24M [00:00<00:00, 76.6MB/s]


In [ ]:
%cd /content/PocketNet
import logging
import os
import sys
import torch
from backbones.augment_cnn import AugmentCNN 
import backbones.genotypes as gt

from configs.config_PocketNetS128 import config as cfg



genotype = gt.from_str(cfg.genotypes["softmax_casia"])
backbone=AugmentCNN(C=cfg.channel, n_layers=cfg.n_layers, genotype=genotype, stem_multiplier=4,
                       emb=cfg.embedding_size)
backbone.load_state_dict(torch.load("/content/295672backbone.pth"))
backbone.eval()
x = torch.randn([1,3,112,112])
torch.onnx.export(backbone,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "pocketnet.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,
                  input_names = ['data'],   # the model's input names
                  output_names = ['fc1']
                  )

/content/PocketNet
call Factorized
call Factorized


In [ ]:
%cd /content/
!wget -q https://github.com/Tencent/ncnn/releases/download/20220701/ncnn-20220701-ubuntu-1804.zip
!unzip -q ncnn-20220701-ubuntu-1804.zip

import os
os.environ['PATH'] += ":/content/ncnn-20220701-ubuntu-1804/bin"

/content


In [ ]:
%cd /content/PocketNet
!onnx2ncnn \
  pocketnet.onnx \
  pocketnet.param \
  pocketnet.bin

/content/PocketNet


In [ ]:
!apt install vulkan-utils
# https://github.com/Tencent/ncnn/wiki/use-ncnnoptimize-to-optimize-model
!ncnnoptimize \
  pocketnet.param \
  pocketnet.bin \
  pocketnet-opt.param \
  pocketnet-opt.bin \
  65536

Reading package lists... Done
Building dependency tree       
Reading state information... Done
vulkan-utils is already the newest version (1.1.126.0+dfsg1-1~gpu18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
Input layer data without shape info, shape_inference skipped
Input layer data without shape info, estimate_memory_footprint skipped


In [ ]:
import torch
torch.__version__

'1.10.0+cu102'

In [ ]:
!mkdir pocketnet
!cp pocketnet-opt.param pocketnet/pocketnet.param
!cp pocketnet-opt.bin pocketnet/pocketnet.bin

!zip -r pocketnet.zip pocketnet

from google.colab import files
files.download("pocketnet.zip")

mkdir: cannot create directory ‘pocketnet’: File exists
  adding: pocketnet/ (stored 0%)
  adding: pocketnet/pocketnet.param (deflated 86%)
  adding: pocketnet/pocketnet.bin (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>